In [ ]:
import pandas as pd
import ee
import geemap

# integrate GEE
ee.Authenticate()
ee.Initialize()

## Load data, select bands, define forest as pixels with canopy cover ≥ 30%

In [ ]:
# load Hansen dataset
dataset = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')

# bands
tree_cover = dataset.select('treecover2000')
loss_year = dataset.select('lossyear')

# define forest threshold
forest_2000 = tree_cover.gte(30)

## Compute yearly forest loss in % loss and square meters

In [ ]:
# level 2: district level
districts = ee.FeatureCollection('FAO/GAUL/2015/level2') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Zambia'))

# baseline forest area per province
def compute_baseline(feature):
    name = feature.get('ADM2_NAME')
    # multiply forest mask by pixel area and rename band
    forest_area = forest_2000.multiply(ee.Image.pixelArea()).rename('forest_area')
    # compute sum of forested area in m² within the district
    stats = forest_area.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = feature.geometry(),
        scale = 30,
        maxPixels = 1e13
    )
    area_m2 = stats.get('forest_area')
    return feature.set({'district': name, 'baseline_m2': area_m2})

baseline = districts.map(compute_baseline).getInfo()

# store baseline forest area per district
baseline_dict = {
    f['properties']['district']: f['properties']['baseline_m2'] or 0 for f in baseline['features']
}

# initialize previous forest area tracker
previous_forest_m2 = {d: baseline_dict.get(d, 0) for d in baseline_dict}
cumulative_loss_m2 = {}

# create empty list to store data
results = []

for year in range(2001, 2023):
    year_offset = year - 2000
    loss_in_year = loss_year.eq(year_offset)
    # mask to forested pixels in loss year
    loss_year_forest = loss_in_year.And(forest_2000)
    # convert to area (square meters)
    loss_area = loss_year_forest.multiply(ee.Image.pixelArea())
    loss_area = loss_area.rename('loss_area')
    # get total loss area for each district
    def compute_loss(feature):
        # district names
        name = feature.get('ADM2_NAME')
        stats = loss_area.reduceRegion(
            reducer = ee.Reducer.sum(),
            geometry = feature.geometry(),
            scale = 30,
            maxPixels = 1e13
        )
       
        area_m2 = stats.get('loss_area')
        return feature.set({
            'district': name, 
            'year': year,
            'loss_m2': area_m2
        })
        
    yearly_results = districts.map(compute_loss).getInfo()

    # extract data to Python
    for feat in yearly_results['features']:
        properties = feat['properties']
        district = properties['district']
        loss_m2 = properties['loss_m2'] or 0
        previous_m2 = previous_forest_m2.get(district, 0)
        cumulative_loss_m2[district] = cumulative_loss_m2.get(district, 0) + loss_m2
        remaining_m2 = previous_m2 - loss_m2
        remaining_ha = max(remaining_m2 / 10000, 0)
        # find annual percent forest loss
        percent_loss_annual = (loss_m2 / previous_m2 * 100) if previous_m2 > 0 else 0 
        previous_forest_m2[district] = remaining_m2
        results.append({
            'district': properties['district'],
            'year': properties['year'],
            'forest_cover_ha': remaining_ha,
            'loss_m2': loss_m2,
            'percent_loss_annual': percent_loss_annual
        })

# convert to pandas dataframe
df = pd.DataFrame(results)
df = df.sort_values(by=['district', 'year'])

# save as csv
df.to_csv('district_forest_cover.csv', index=False)